In [20]:
# BEFORE RUNNING THE CELLS, PLEASE GO TO "EDIT" -> "NOTEBOOK SETTINGS" -> AND SET "HARDWARE ACCELERATOR" TO GPU

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import numpy as np 
import matplotlib.pyplot as plt 
import argparse
import os
import cv2 
import random

import sys
from PIL import Image

import pickle

import math



In [6]:
!unzip olddata.zip


Archive:  olddata.zip
   creating: olddata/Boring/
   creating: olddata/Boring/combined/
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample5_100.png  
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample5_101.png  
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample5_102.png  
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample5_103.png  
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample5_104.png  
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample5_105.png  
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample5_106.png  
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample5_107.png  
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample5_108.png  
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample5_109.png  
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample5_110.png  
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample5_111.png  
  inflating: olddata/Boring/S01_B0256_States2_Boring_Sample

In [30]:
DATADIR = "./olddata" 
CATEGORIES = ["Boring", "Interesting"]
training_data = []
IMG_SIZE = 1185 # Hyperparameter. calculated to be stitched together size. each img is 396 x 396. Stitching them together 396*3 = 1188x 1188. So 1185 is an approximation of that.

In [8]:
## Stitch the images together 
## 0 1 2 
## 3 4 5
## 6 7 8

def stitch_images(file_path, file_name):
    images = [Image.open(image) for image in [file_path + "/" + file_name + str(x) + ".png" for x in range(100, 109)]]
    widths, heights = zip(*(i.size for i in images))
    total_width = int(sum(widths) / 3)
    total_height = int(sum(heights) / 3)
    new_image = Image.new("RGB", (total_width, total_height))
    for index in range(0, 9):
        image = images[index]
        new_image.paste(image, ((index % 3) * image.size[0], math.floor(index / 3) * image.size[1]))

    IMAGE_DIR = os.path.join(file_path, "combined/") + file_name + "combined.png"
    new_image.save(IMAGE_DIR)
    return IMAGE_DIR

In [9]:
def create_training_data():
    for category in CATEGORIES: 
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for image in sorted(os.listdir(path)): 
            if "100" not in image: ## Find the starting frame
                continue
            IMAGE_DIR = stitch_images(path, image[0:-7])
            try: 
                img_array = cv2.imread(IMAGE_DIR, cv2.IMREAD_GRAYSCALE) #cv2.IMREAD_GRAYSCALE: It specifies to load an image in grayscale mode. Alternatively, we can pass integer value 0 for this flag.
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e: 
                pass 

In [10]:
create_training_data() 
random.shuffle(training_data) 

In [11]:
X, y = [], []
for features, label in training_data: 
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

In [12]:
# Save Data 
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [13]:
## Load saved Data
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

In [29]:
#print(X[0][0].flatten()[len(X[0][0].flatten())//2])
print(y)

[0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1 1 0 1 1 1
 1 0 1 0 1 0 1]


In [14]:
## Data Training 
### Data Normalization

# A lot of hyperparameters here, e.g. # of layers, neurons, etc.

X = X / 255.0 
model = keras.Sequential()
model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# batch_size should be tuned

In [18]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
tf.test.gpu_device_name()

'/device:GPU:0'

In [19]:
model.fit(X, y, batch_size=2, epochs=10) 

Epoch 1/10
22/22 [==============================] - 4s 178ms/step - loss: 40.5163 - accuracy: 0.3864
Epoch 2/10
22/22 [==============================] - 4s 182ms/step - loss: 0.3294 - accuracy: 0.9091
Epoch 3/10
22/22 [==============================] - 4s 184ms/step - loss: 0.1252 - accuracy: 0.9545
Epoch 4/10
22/22 [==============================] - 4s 187ms/step - loss: 0.0505 - accuracy: 0.9773
Epoch 5/10
22/22 [==============================] - 4s 187ms/step - loss: 0.0425 - accuracy: 0.9773
Epoch 6/10
22/22 [==============================] - 4s 188ms/step - loss: 0.0397 - accuracy: 0.9773
Epoch 7/10
22/22 [==============================] - 4s 188ms/step - loss: 0.1313 - accuracy: 0.9318
Epoch 8/10
22/22 [==============================] - 4s 190ms/step - loss: 1.1635 - accuracy: 0.9773
Epoch 9/10
22/22 [==============================] - 4s 191ms/step - loss: 0.0311 - accuracy: 1.0000
Epoch 10/10
22/22 [==============================] - 4s 189ms/step - loss: 0.0293 - accuracy: 1.000